In [ ]:
!pip install pyedflib

     |████████████████████████████████| 1.2MB 5.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyedflib: filename=pyEDFlib-0.1.19-cp36-cp36m-linux_x86_64.whl size=925679 sha256=035f4ccbaf28bd831f6fb8cba282eaae15e63ea4baf7d6e9ba8617244e32bb36
  Stored in directory: /root/.cache/pip/wheels/1f/b7/24/a565e4f7471617165f1b040651b52d87ad1885aaf32e02d4f9
Successfully built pyedflib


In [ ]:
import pyedflib as edf
import numpy as np
import pandas as pd
import pywt
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import subprocess as sp
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/chb_dataset/'
raw_data_path = "/content/raw_data/"
result_data_path = drive_path + "processed_data/"

if not os.path.exists(raw_data_path):
  os.mkdir(raw_data_path)

Mounted at /content/drive


In [ ]:
# use the linux command 'mv' to move files from colab storage to your drive

In [ ]:
#for downloading files
def download_files(patient_num, file_num_list):
  for file_num in file_num_list:
    file_name = f'chb{str(patient_num).zfill(2)}_{str(file_num).zfill(2)}.edf'
    #file_name = f'chb{patient_num}b_{file_num}.edf'#only for patient 17
    url = f'https://physionet.org/static/published-projects/chbmit/1.0.0/chb{str(patient_num).zfill(2)}/'+file_name
    response_code = sp.call(['wget', url])
    if(response_code != 0):
      print(file_name+"  Not found")
    dest_path = raw_data_path + file_name
    response_code = sp.call(['mv', file_name, dest_path])
    if(response_code == 0):
      print("Downloaded "+file_name)

Data processing using 18 channels

In [ ]:
#function to join two files
def join_data(file_one, file_two):
    file_one = edf.EdfReader(raw_data_path + file_one)
    file_two = edf.EdfReader(raw_data_path + file_two)

    file_one_max, file_one_min = file_one.getPhysicalMaximum(), file_one.getPhysicalMinimum()
    file_two_max, file_two_min = file_two.getPhysicalMaximum(), file_two.getPhysicalMinimum()

    joined_channels = list()
    for i in range(18):
      partOne = (file_one.readSignal(i) - file_one_min[i]) / (file_one_max[i] - file_one_min[i])
      partTwo = (file_two.readSignal(i) - file_two_min[i]) / (file_two_max[i] - file_two_min[i])
      joined = np.concatenate((partOne, partTwo))
      joined_channels.append(joined)
    joined_channels = np.array(joined_channels)
    return joined_channels

In [ ]:
#create data using 18 channels
def process_data(file_list, start_times, preictal_filename, interictal_filename):
    preictal_data, interictal_data = list(), list()
    preictal_window = interictal_window = PREICTAL_INTERVAL_IN_MINS * 60 * 256

    for file, start_time in zip(file_list, start_times):
        print(f"Processing file: {file}")
        inter_raw_data = edf.EdfReader(raw_data_path + file[0])
        if type(file[1]) == list:
            pre_data = join_data(file[1][0], file[1][1])
        else:
            pre_raw_data = edf.EdfReader(raw_data_path + file[1])
            pre_data = np.array([pre_raw_data.readSignal(i) for i in range(18)])
            
            #normalize pre_data
            max_value = np.array(pre_raw_data.getPhysicalMaximum()[0:18]).reshape(-1, 1)
            min_value = np.array(pre_raw_data.getPhysicalMinimum()[0:18]).reshape(-1, 1)
            pre_data = (pre_data - min_value) / (max_value - min_value)

            pre_raw_data._close()
            del pre_raw_data

        inter_data = np.array([inter_raw_data.readSignal(i) for i in range(18)])

        #normalize inter_data
        max_value = np.array(inter_raw_data.getPhysicalMaximum()[0:18]).reshape(-1, 1)
        min_value = np.array(inter_raw_data.getPhysicalMinimum()[0:18]).reshape(-1, 1)
        inter_data = (inter_data - min_value) / (max_value - min_value)

        inter_raw_data._close()
        del inter_raw_data
        
        pre_data = pre_data[:, 0:start_time[1] * 256]
        
        preictal_start = pre_data.shape[1] - preictal_window
        print(f"preictal start: {preictal_start}")
        interictal_start = np.random.random_integers(low = 0, high = inter_data.shape[1] - interictal_window)
        interictal_end = interictal_start + interictal_window

        preictal = pre_data[:, preictal_start:]
        interictal = inter_data[:, interictal_start : interictal_end]
        preictal_data.append(preictal)
        interictal_data.append(interictal)

    pre_data, inter_data  = np.array(preictal_data), np.array(interictal_data)
    print(f"Preictal File shape: {pre_data.shape}")
    print(f"Interictal File shape: {inter_data.shape}")
    
    np.save(result_data_path + preictal_filename, pre_data)
    print(f"Saved {preictal_filename}")
    np.save(result_data_path + interictal_filename, inter_data)
    print(f"Saved {interictal_filename}")

patient 1

In [ ]:
data = [('chb01_02.edf', 'chb01_03.edf'), 
        ('chb01_05.edf', ['chb01_03.edf','chb01_04.edf']), 
        ('chb01_12.edf', ['chb01_14.edf','chb01_15.edf']), 
        ('chb01_17.edf', ['chb01_15.edf', 'chb01_16.edf']), 
        ('chb01_22.edf', ['chb01_17.edf','chb01_18.edf']),
        ('chb01_25.edf', ['chb01_20.edf', 'chb01_21.edf'])]

start_times = [(0, 2996), 
               (0, 1467 + 3600), 
               (0, 1732 + 3600), 
               (0, 1015 + 3600), 
               (0, 1720 + 3600), 
               (0, 327 + 3600)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 1

file_list = [2, 3, 4, 5, 12, 14, 15, 16, 17, 18, 20, 21, 22, 25]
download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 2

In [ ]:
data = [('chb02_01.edf', ['chb02_15.edf', 'chb02_16.edf']), 
        ('chb02_17.edf', 'chb02_16+.edf'),
        ('chb02_20.edf', 'chb02_19.edf')]

start_times = [(0, 3600 + 130), 
               (0, 2972),
               (0, 3369)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 2

file_list = ['1', '15', '16', '16+', '17', '19', '20']
download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 3

In [ ]:
data = [('chb03_06.edf', ['chb03_02.edf', 'chb03_03.edf']), 
        ('chb03_20.edf', 'chb03_04.edf'),
        ('chb03_33.edf', 'chb03_34.edf'),
        ('chb03_37.edf', 'chb03_35.edf'),
        ('chb03_38.edf', ['chb03_35.edf','chb03_36.edf'])]

start_times = [(0, 3600 + 432),
               (0, 2162), 
               (0, 1982),
               (0, 2592),
               (0, 3600 + 1725)]

file_list = [2, 3, 4, 6, 20, 33 ,34, 35, 36, 37, 38]
PATIENT_NUM = 3
PREICTAL_INTERVAL_IN_MINS = 20

#download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 4

In [ ]:
data = [('chb04_01.edf', 'chb04_05.edf'),
        ('chb04_07.edf', 'chb04_08.edf'),
        ('chb04_34.edf', ['chb04_27.edf','chb04_28.edf']),
        ('chb04_25.edf', 'chb04_28.edf')
        ]

start_times = [(0, 7804),
               (0, 6446),
               (0, 1679+1781),
               (0, 3782)]

file_list = [1, 5, 7, 8, 34, 27,  28, 25]

PATIENT_NUM = 4
PREICTAL_INTERVAL_IN_MINS = 30

#download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 5

In [ ]:

data = [('chb05_01.edf', ['chb05_05.edf', 'chb05_06.edf']),
        ('chb05_27.edf', ['chb05_12.edf', 'chb05_13.edf']),
        ('chb05_14.edf', 'chb05_16.edf'),
        ('chb05_02.edf', 'chb05_17.edf'),
        ('chb05_10.edf', 'chb05_22.edf')
        ]

start_times = [(0, 3600+417),
               (0, 3600+1086),
               (0, 2317),
               (0, 2451),
               (0, 2348)]
file_list = [1, 5, 6, 27, 12, 13, 14, 16, 2, 17, 10, 22]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 5

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 6

In [ ]:
data = [('chb06_02.edf', 'chb06_01.edf'),
        ('chb06_03.edf','chb06_01.edf'),
        ('chb06_07.edf', 'chb06_04.edf'),
        ('chb06_16.edf', 'chb06_09.edf'),
        ('chb06_15.edf', 'chb06_10.edf'),
        ('chb06_17.edf', ['chb06_12.edf', 'chb06_13.edf']),
        ('chb06_08.edf', 'chb06_18.edf'),
        ('chb06_14.edf', 'chb06_24.edf')]

start_times = [(0, 7461),
               (0, 13525),
               (0, 6211),
               (0, 12500),
               (0, 10833),
               (0, 3600+506),
               (0, 7799),
               (0, 9387)]

file_list = [1, 2, 3, 4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 24]

PREICTAL_INTERVAL_IN_MINS = 20
PATIENT_NUM = 6

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 7

In [ ]:
data = [('chb07_01.edf','chb07_12.edf'),
        ('chb07_10.edf', 'chb07_13.edf'),
        ('chb07_18.edf', 'chb07_19.edf')]

start_times = [(0, 4920),
               (0, 3285),
               (0, 13668)]

file_list = [1, 10, 12, 13, 18, 19]

PATIENT_NUM = 7
PREICTAL_INTERVAL_IN_MINS = 30

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 8

In [ ]:
data = [('chb08_04.edf','chb08_02.edf'),
        ('chb08_03.edf', 'chb08_05.edf'),
        ('chb08_10.edf', 'chb08_11.edf'),
        ('chb08_12.edf', 'chb08_13.edf'),
        ('chb08_20.edf', 'chb08_21.edf')]

start_times = [(0, 2670),
               (0, 2856),
               (0, 2988),
               (0, 2417),
               (0, 2083)]

file_list = [2, 3, 4, 5, 10, 11, 12, 13, 20, 21]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 8

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 9

In [ ]:
data = [('chb09_05.edf','chb09_06.edf'),
        ('chb09_07.edf', 'chb09_08.edf'),#1st seizure  in file 
        ('chb09_10.edf', 'chb09_08.edf'),#2nd seizure in file
        ('chb09_18.edf', 'chb09_19.edf')]

start_times = [(0, 12231),
               (0, 2951),
               (0, 9196),
               (0, 5299)]

file_list = [5, 6, 7, 8, 10, 18, 19]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 9

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 10

In [ ]:
data = [('chb10_08.edf', 'chb10_12.edf'),
        ('chb10_19.edf', 'chb10_20.edf'),
        ('chb10_02.edf', 'chb10_27.edf'),
        ('chb10_28.edf', 'chb10_30.edf'),
        ('chb10_22.edf', 'chb10_31.edf'),
        ('chb10_01.edf', 'chb10_38.edf')]

start_times = [(0, 6313),
               (0, 6888),
               (0, 2381),
               (0, 3021),
               (0, 3801),
               (0, 4618)]

file_list = [1, 2, 8, 12, 19, 20, 22, 27, 28, 30, 31, 38]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 10

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 11

In [ ]:

data = [('chb11_01.edf', ['chb11_92.edf','chb11_99.edf']),
        ('chb11_56.edf', 'chb11_92.edf'),
        ('chb11_55.edf', ['chb11_63.edf','chb11_82.edf'])]

start_times = [(0, 3600+1454),
               (0, 2695),
               (0, 3600+298)]

PATIENT_NUM = 11
PREICTAL_INTERVAL_IN_MINS = 30

file_list = [1, 55, 56, 50, 63, 82, 92, 99]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 12

In [ ]:
#some files not present
data = [('chb12_24.edf', 'chb12_09.edf'),
        ('chb12_34.edf', ['chb12_10.edf', 'chb12_11.edf']),
        ('chb12_40.edf', ['chb12_21.edf', 'chb12_23.edf']),
        ('chb12_32.edf', 'chb12_33.edf'),
        ('chb12_20.edf', ['chb12_35.edf', 'chb12_36.edf']),
        ('chb12_19.edf', ['chb12_37.edf', 'chb12_38.edf']),
        ('chb12_39.edf', ['chb12_41.edf', 'chb12_42.edf'])]

start_times = [(0, 3082),
               (0, 3600 + 1085),
               (0, 3600 + 253),
               (0, 2185),
               (0, 3600 + 653),
               (0, 3600 + 1548),
               (0, 3600 + 699)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 12

flie_list = [9, 10, 11, 19, 20, 21, 23, 24, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 13

In [ ]:
data = [('chb13_15.edf','chb13_19.edf'),
        ('chb13_16.edf', ['chb13_18.edf','chb13_21.edf']),
        ('chb13_38.edf', ['chb13_39.edf','chb13_40.edf']),
        ('chb13_47.edf', 'chb13_55.edf'),
        ('chb13_13.edf', 'chb13_58.edf'),
        ('chb13_12.edf', 'chb13_59.edf'),
        ('chb13_11.edf', ['chb13_56.edf','chb13_60.edf'])]

start_times = [(0, 2077),
               (0, 3600 + 934),
               (0, 3600 + 142),
               (0, 2436),
               (0, 2474),
               (0, 3339),
               (0, 3600 + 638)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 13

file_list = [11, 12, 13, 15, 16, 18, 19, 21, 38, 39, 40, 47, 55, 56, 58, 59, 60]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 14

In [ ]:
data = [('chb14_02.edf', 'chb14_03.edf'),
        ('chb14_07.edf', ['chb14_01.edf','chb14_04.edf']),
        ('chb14_13.edf', 'chb14_06.edf'),
        ('chb14_14.edf', 'chb14_11.edf'),
        ('chb14_16.edf', 'chb14_17.edf'),
        ('chb14_19.edf', 'chb14_27.edf')]

start_times = [(0, 1986),
               (0, 3600 + 1372),
               (0, 1911),
               (0, 1838),
               (0, 3239),
               (0, 2833)]

PREICTAL_INTERVAL_IN_MINS = 20
PATIENT_NUM = 14

file_list = ["01", "02", "03", "04", "06", "07", "11", "13", "14", "16", "17", "19", "27"]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 15

In [ ]:
data = [('chb15_04.edf', ['chb15_09.edf', 'chb15_10.edf']),
        ('chb15_13.edf', ['chb15_14.edf', 'chb15_15.edf']),
        ('chb15_16.edf', 'chb15_17.edf'),
        ('chb15_32.edf', ['chb15_30.edf', 'chb15_31.edf']),
        ('chb15_45.edf', 'chb15_46.edf')]

start_times = [(0, 3600 + 1082),
               (0, 3600 + 1591),
               (0, 1925),
               (0, 3600 + 1751),
               (0, 3322)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 15

file_list = [4, 9, 10, 13, 14, 15, 16, 17, 30, 31, 32, 45, 46]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 16

In [ ]:
data = [('chb16_09.edf', 'chb16_10.edf'),
        ('chb16_04.edf', ['chb16_10.edf', 'chb16_11.edf']),
        ('chb16_09.edf', 'chb16_14.edf'),
        ('chb16_13.edf', ['chb16_15.edf', 'chb16_16.edf']),
        ('chb16_19.edf', ['chb16_13.edf', 'chb16_17.edf'])]

start_times = [(0, 2290),
               (0, 3600 + 1120),
               (0, 1854),
               (0, 3600 + 1214),
               (0, 3600 + 227)]

PREICTAL_INTERVAL_IN_MINS = 20
PATIENT_NUM = 16

file_list = [4, 9, 10, 11, 13, 14, 15, 16, 17, 19]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 17

In [ ]:
!wget https://physionet.org/static/published-projects/chbmit/1.0.0/chb17/chb17b_63.edf

In [ ]:
data = [('chb17a_05.edf', 'chb17a_03.edf'),
        ('chb17a_06.edf', 'chb17a_04.edf'),
        ('chb17c_13.edf', 'chb17b_63.edf')]

start_times = [(0, 2282),
               (0, 3025),
               (0, 3136)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 17

file_list = ["03", "04", "05", "06", "13", "63"]

#download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 18

In [ ]:
#didnt choose chb18_30.edf, chb18_36.edf as they are small
data = [('chb18_05.edf', 'chb18_29.edf'),
        ('chb18_16.edf', 'chb18_31.edf'),
        ('chb18_26.edf', 'chb18_32.edf'),
        ('chb18_34.edf', 'chb18_35.edf')]

start_times = [(0, 3477),
               (0, 2087),
               (0, 1908),
               (0, 2196)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 18

file_list = [5, 16, 26, 29, 31, 32, 34, 35]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 19

In [ ]:
data = [('chb19_01.edf', ['chb19_27.edf', 'chb19_28.edf']),
        ('chb19_10.edf', 'chb19_29.edf'),
        ('chb19_25.edf', 'chb19_30.edf')]

start_times = [(0, 3600 + 299),
               (0, 2964),
               (0, 3159)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 19

file_list = [1, 10, 25, 27, 28, 29, 30]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 20

In [ ]:
data = [('chb20_01.edf', ['chb20_11.edf', 'chb20_12.edf']),
        ('chb20_17.edf', ['chb20_12.edf', 'chb20_13.edf']),
        ('chb20_59.edf', 'chb20_14.edf'),
        ('chb20_21.edf', 'chb20_16.edf'),
        ('chb20_27.edf', ['chb20_60.edf', 'chb20_68.edf'])]

start_times = [(0, 3600 + 94),
               (0, 3600 + 1440),
               (0, 1971),
               (0, 2226),
               (0, 3600 + 1393)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 20

file_list = [1, 11, 12, 13, 14, 16, 17, 21, 27, 59, 60, 68]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 21

In [ ]:
data = [('chb21_01.edf', ['chb21_18.edf', 'chb21_19.edf']),
        ('chb21_14.edf', 'chb21_20.edf'),
        ('chb21_25.edf', 'chb21_21.edf'),
        ('chb21_33.edf', 'chb21_22.edf')]

start_times = [(0, 3600 + 1288),
               (0, 2627),
               (0, 2003),
               (0, 2553)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 21

file_list = [1, 14, 18, 19, 20, 21, 22, 25, 33]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 22

In [ ]:
data = [('chb22_01.edf', 'chb22_20.edf'),
        ('chb22_77.edf', 'chb22_25.edf'),
        ('chb22_17.edf', ['chb22_30.edf', 'chb22_38.edf'])]

start_times = [(0, 3367),
               (0, 3139),
               (0, 3600 + 1263)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 22

file_list = [1, 17, 20, 25, 30, 38, 77]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 23

In [ ]:
data = [('chb23_10.edf', 'chb23_06.edf'),
        ('chb23_20.edf', ['chb23_07.edf', 'chb23_08.edf']),
        ('chb23_16.edf', 'chb23_09.edf'),
        ('chb23_17.edf', 'chb23_09.edf'),
        ('chb23_19.edf', 'chb23_09.edf')]

start_times = [(0, 3962),
               (0, 3600 + 325),
               (0, 2589),
               (0, 6885),
               (0, 8505)]

PREICTAL_INTERVAL_IN_MINS = 30
PATIENT_NUM = 23

file_list = [10, 6, 7, 8, 9, 16, 17, 19, 20]

download_files(PATIENT_NUM, file_list)

interictal_name = f"patient_{PATIENT_NUM}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{PATIENT_NUM}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
process_data(data, start_times, preictal_name, interictal_name)

patient 24

In [ ]:
#NOT USED
PREICTAL_INTERVAL_IN_MINS = 30
patient_num = 24

data = []

start_times = [(0, 3962),
               (0, 3600 + 325),
               (0, 2589),
               (0, 6885),
               (0, 8505)]

file_num_list = ["10", "06", "07", "08", "09", "16", "17", "19", "20"]

download_files(patient_num, file_num_list)

interictal_name = f"patient_{patient_num}_inter_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
preictal_name = f"patient_{patient_num}_pre_interval_{PREICTAL_INTERVAL_IN_MINS}.npy"
to_table_2(data, start_times, preictal_name, interictal_name)